In [1]:
from utils import *

# Method 1: MANUAL EXTRACTION FROM GPCRdb

In [4]:
pdbs = getpdbs()

In [5]:
print(pdbs)

['data/structures/pdb_structures/adrb2_human_7DHR.pdb', 'data/structures/pdb_structures/adrb1_melga_7JJO.pdb', 'data/structures/pdb_structures/pe2r4_human_7D7M.pdb', 'data/structures/pdb_structures/adrb2_human_6E67.pdb', 'data/structures/pdb_structures/adrb2_human_7BZ2.pdb', 'data/structures/pdb_structures/adrb2_human_3SN6.pdb', 'data/structures/pdb_structures/aa2ar_human_5G53.pdb', 'data/structures/pdb_structures/adrb2_human_7DHI.pdb', 'data/structures/pdb_structures/gpbar_human_7CFM.pdb', 'data/structures/pdb_structures/gpr52_human_6LI3.pdb', 'data/structures/pdb_structures/gpbar_human_7CFN.pdb', 'data/structures/pdb_structures/aa2ar_human_6GDG.pdb', 'data/structures/pdb_structures/gpbar_human_7BW0.pdb', 'data/structures/pdb_structures/adrb2_human_6NI3.pdb']


In [12]:
prots = list(set([x[-20:-9] for x in pdbs]))
print(prots)

['gpr52_human', 'adrb2_human', 'pe2r4_human', 'adrb1_melga', 'aa2ar_human', 'gpbar_human']


In [13]:
from urllib.request import urlopen
import json

# fetch a protein
def get_seq(prot):
    url = 'https://gpcrdb.org/services/protein/' + prot + '/'
    response = urlopen(url)
    protein_data = json.loads(response.read().decode('utf-8'))
    print(protein_data)
    return(protein_data['sequence'])

In [14]:
for p in prots:
    seq = get_seq(p)
    print()

{'entry_name': 'gpr52_human', 'name': '<i>GPR52</i>', 'accession': 'Q9Y2T5', 'family': '001_011_001_028', 'species': 'Homo sapiens', 'source': 'SWISSPROT', 'residue_numbering_scheme': 'GPCRdb(A)', 'sequence': 'MNESRWTEWRILNMSSGIVNVSERHSCPLGFGHYSVVDVCIFETVVIVLLTFLIIAGNLTVIFVFHCAPLLHHYTTSYFIQTMAYADLFVGVSCLVPTLSLLHYSTGVHESLTCQVFGYIISVLKSVSMACLACISVDRYLAITKPLSYNQLVTPCRLRICIILIWIYSCLIFLPSFFGWGKPGYHGDIFEWCATSWLTSAYFTGFIVCLLYAPAAFVVCFTYFHIFKICRQHTKEINDRRARFPSHEVDSSRETGHSPDRRYAMVLFRITSVFYMLWLPYIIYFLLESSRVLDNPTLSFLTTWLAISNSFCNCVIYSLSNSVFRLGLRRLSETMCTSCMCVKDQEAQEPKPRKRANSCSI', 'genes': ['GPR52', 'ECO:0000312|HGNC:HGNC:4508}']}

{'entry_name': 'adrb2_human', 'name': '&beta;<sub>2</sub>-adrenoceptor', 'accession': 'P07550', 'family': '001_001_003_008', 'species': 'Homo sapiens', 'source': 'SWISSPROT', 'residue_numbering_scheme': 'GPCRdb(A)', 'sequence': 'MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSLACADLVMGLAVVPFGAAHILMKMWTFGNFWCEFWTSIDVLCVTASIETLCVIAVDRYFAITSPFKYQSLL

In [ ]:
# Get secondary structure

In [11]:
# Get angles

# Method 2: Uniprot list

In [2]:
data = getuniprot()

In [3]:
data

,gene,id,name
0,ACM1_HUMAN,P11229,Muscarinic acetylcholine M1 [CHRM1]
1,ACM2_HUMAN,P08172,Muscarinic acetylcholine M2 [CHRM2]
2,ACM3_HUMAN,P20309,Muscarinic acetylcholine M3 [CHRM3]
3,ACM4_HUMAN,P08173,Muscarinic acetylcholine M4 [CHRM4]
4,ACM5_HUMAN,P08912,Muscarinic acetylcholine M5 [CHRM5]
...,...,...,...
832,FZD8_HUMAN,Q9H461,Frizzled-8 (Fz-8) (hFz8) [FZD8]
833,FZD9_HUMAN,O00144,Frizzled-9 (Fz-9) (hFz9) (FzE6) (CD349 antigen...
834,FZD10_HUMAN,Q9ULW2,Frizzled-10 (Fz-10) (hFz10) (FzE7) (CD350 ant...
835,SMO_HUMAN,Q99835,Smoothened homolog (SMO) [SMO; SMOH]


In [4]:
id_list = data['id'].tolist()
q = ''
for _ in id_list:
    q+=_+' '

In [5]:
import urllib
code = "Q7Z7W5"
data = urllib.urlopen("http://www.uniprot.org/uniprot/" + code + ".txt").read()

AttributeError: module 'urllib' has no attribute 'urlopen'

In [10]:
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

params = {
'from': 'ACC+ID',
'to': 'ENSEMBL_ID',
'format': 'tab',
'query': q
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
    response = f.read()
print(response.decode('utf-8'))

From	To
P11229	ENSG00000168539
P08172	ENSG00000181072
P20309	ENSG00000133019
P08173	ENSG00000180720
P08912	ENSG00000184984
P30542	ENSG00000163485
P29274	ENSG00000128271
P29275	ENSG00000170425
P0DMS8	ENSG00000282608
P47900	ENSG00000169860
P41231	ENSG00000175591
P51582	ENSG00000186912
Q15077	ENSG00000171631
Q86VZ1	ENSG00000182162
O00398	ENSG00000078589
Q96G91	ENSG00000244165
Q9H244	ENSG00000169313
Q9BPV8	ENSG00000181631
Q15391	ENSG00000174944
Q9BXC1	ENSG00000147138
Q15722	ENSG00000213903
Q15722	ENSG00000285456
Q9NPC1	ENSG00000213906
Q9NPC1	ENSG00000285203
P35348	ENSG00000120907
P35368	ENSG00000170214
P25100	ENSG00000171873
P08913	ENSG00000150594
P18089	ENSG00000274286
P18825	ENSG00000184160
P08588	ENSG00000043591
P07550	ENSG00000169252
P13945	ENSG00000188778
P30556	ENSG00000144891
P50052	ENSG00000180772
P35414	ENSG00000134817
Q14330	ENSG00000125245
Q8TDU6	ENSG00000179921
P28336	ENSG00000135577
P30550	ENSG00000126010
P32247	ENSG00000102239
P46663	ENSG00000100739
P30411	ENSG00000168398
P21

In [8]:
from Bio import SeqIO


uniprot = SeqIO.index("uniprot_sprot.dat", "swiss")
with open("selected.dat", "wb") as out_handle:
    for acc in ["P33487", "P19801", "P13689", "Q8JZQ5", "Q9TRC7"]:
        out_handle.write(uniprot.get_raw(acc))

FileNotFoundError: [Errno 2] No such file or directory: 'uniprot_sprot.dat'

## PDB structure query

In [ ]:
# 1 map uniprot id to pdb id

In [ ]:
# check if structure exists

In [ ]:
# check if docking / complex structure exists

## Secondary structure ID / Alignment

# Remark: PRECOG